### **Pregunta** 1

  a) Por la deficion de ambos sesgos, el sesgo inductivo esta incluido en el algoritmo es producido en base al diseño del programador para definir la forma en que podría distribuir los datos con base a los sabido previamente. Este tipo de sesgos es irrediucible. En cambio el sesgo de una variable esta dado por como fueron genereados esos datos.

  b) El sesgo dentro de k-NN, conlleva a que tan cercanos esta las predicciones dentro de todas las predicciones con los valores reales, en este caso, conlleva que tan bien se categoriza un punto en el espacio. Para el caso de los arboles de desición en base a la ganancia de información, el sesgo esta en base a que tan baja es el error para definir la clasificación de cierto dato. Además de que tan bajo es la entreopia de 

  c)

In [2]:
import sys
sys.setrecursionlimit(99999)
headers = ['Outlook', 'Temperature', 'Humidity', 'Wind', 'Plays']
data=[['Sunny','Hot','High','Weak','No'],
      ['Sunny','Hot','High','Strong','No'],
      ['Overcast','Hot','High','Weak','Yes'],
      ['Rain','Mild','High','Weak','Yes'],
      ['Rain','Cool','Normal','Weak','Yes'],
      ['Rain','Cool','Normal','Strong','No'],
      ['Overcast','Cool','Normal','Strong','Yes'],
      ['Sunny','Mild','High','Weak','No'],
      ['Sunny','Cool','Normal','Weak','Yes'],
      ['Rain','Mild','Normal','Weak','Yes'],
      ['Sunny','Mild','Normal','Strong','Yes'],
      ['Overcast','Mild','High','Strong','Yes'],
      ['Overcast','Hot','Normal','Weak','Yes'],
      ['Rain','Mild','High','Strong','No']]

In [3]:
class decision_node:
    def __init__(self,col=-1, value = '', branches=None, results=None):
        self.col=col
        self.value = value
        self.results=results
        self.branches = branches

In [4]:
def divide_set(rows,column):
    sets = dict()
    for row in rows:
        if row[column] not in sets.keys():
            sets[row[column]] = []
        sets[row[column]].append(row)
    return sets
divide_set(data,0)

{'Sunny': [['Sunny', 'Hot', 'High', 'Weak', 'No'],
  ['Sunny', 'Hot', 'High', 'Strong', 'No'],
  ['Sunny', 'Mild', 'High', 'Weak', 'No'],
  ['Sunny', 'Cool', 'Normal', 'Weak', 'Yes'],
  ['Sunny', 'Mild', 'Normal', 'Strong', 'Yes']],
 'Overcast': [['Overcast', 'Hot', 'High', 'Weak', 'Yes'],
  ['Overcast', 'Cool', 'Normal', 'Strong', 'Yes'],
  ['Overcast', 'Mild', 'High', 'Strong', 'Yes'],
  ['Overcast', 'Hot', 'Normal', 'Weak', 'Yes']],
 'Rain': [['Rain', 'Mild', 'High', 'Weak', 'Yes'],
  ['Rain', 'Cool', 'Normal', 'Weak', 'Yes'],
  ['Rain', 'Cool', 'Normal', 'Strong', 'No'],
  ['Rain', 'Mild', 'Normal', 'Weak', 'Yes'],
  ['Rain', 'Mild', 'High', 'Strong', 'No']]}

In [5]:
def unique_counts(rows):
    results={}
    for row in rows:
        r = row[-1]
        if r not in results: 
            results[r]=0
        results[r] += 1
    return results
unique_counts(data)

{'No': 5, 'Yes': 9}

In [6]:
from math import log2
def entropy(rows):
    results=unique_counts(rows)
    # Now calculate the entropy
    ent=0.0
    for r in results.keys():
        p=float(results[r])/len(rows) 
        ent=ent-p*log2(p)
    return ent
entropy(data)

0.9402859586706311

In [7]:
def getList(dict): 
    list = [] 
    for key in dict.keys(): 
        list.append(key) 
          
    return list

In [28]:
# construcción del árbol usando ganancia de información 
def build_tree(rows, headers, value='', prof=0 ,vistos = []):
    # print('----------------------')
    # print(prof)
    # print(rows)
    # print(headers)
    # print(value)
    # print(vistos)
    if len(rows) == 0: 
        return decision_node(value = value)
    elif len(set([row[-1] for row in rows])) == 1:
        return decision_node(value = value, results=unique_counts(rows))
    current_score = entropy(rows)
    best_gain = 0.0
    best_criteria = None
    best_sets = None
    column_count = len(rows[0]) - 1
    gains = {}
    for col in range(0, column_count):
        if col not in vistos:
            # print("best gain ",best_gain)
            
            sets = divide_set(rows, col)
            gain = current_score
            for key in sets.keys():
                s = float(len(sets[key])) / len(rows)
                gain -= s*entropy(sets[key])
            # print("current gain ",gain)
            gains[gain] = [col,sets]
    # print(gains)
    if len(gains) != 0:
        best_gain = min(getList(gains))
        best_criteria = headers[gains[best_gain][0]]
        best_sets = gains[best_gain][1]
        vistos.append(gains[best_gain][0])
    else:
        best_sets = []
    branches = []
    for s in best_sets:
        lista = vistos[:]
        branches.append(build_tree(best_sets[s], headers, s,prof+1,lista))
    return decision_node(col=best_criteria, value = value, branches=branches)

In [24]:
def print_tree(tree, indent = ''):
    if tree.results != None:
        print(indent + str(tree.results))
    else:
        if tree.value == '':
            print(indent + tree.col + "?")
        for branch in tree.branches:
            to_print = indent + '  ' + branch.value + " -> "
            if branch.col != -1:
                print(to_print + branch.col + "?")
                print_tree(branch, ' '*len(to_print))
            else:
                print(to_print + str(branch.results))

In [29]:
print_tree(build_tree(data, headers))

Temperature?
  Hot -> Wind?
           Weak -> Humidity?
                     High -> Outlook?
                               Sunny -> {'No': 1}
                               Overcast -> {'Yes': 1}
                     Normal -> {'Yes': 1}
           Strong -> {'No': 1}
  Mild -> Wind?
            Weak -> Humidity?
                      High -> Outlook?
                                Rain -> {'Yes': 1}
                                Sunny -> {'No': 1}
                      Normal -> {'Yes': 1}
            Strong -> Humidity?
                        Normal -> {'Yes': 1}
                        High -> Outlook?
                                  Overcast -> {'Yes': 1}
                                  Rain -> {'No': 1}
  Cool -> Humidity?
            Normal -> Wind?
                        Weak -> {'Yes': 2}
                        Strong -> Outlook?
                                    Rain -> {'No': 1}
                                    Overcast -> {'Yes': 1}


In [ ]:
podemos decir que el sesgo inductivo es menor en este caso, ya que al saber menos información el modelo a medida que predice, tiene que seguir recorriendo en las ramas para llegar a una clasificación. Yo creo que no, ya que en el arbol de desición tradicional tenemos una ganacia de información, aun cuando no ocupa eso, mayor a niveles más altos que este, ya que este ocupa las peores ganacias.

### **Pregunta** 2

  a) 


In [1]:

# Code source: Gaël Varoquaux
# Modified for documentation by Jaques Grobler
# License: BSD 3 clause
# sacado de https://scikit-learn.org/stable/auto_examples/linear_model/plot_iris_logistic.html#sphx-glr-auto-examples-linear-model-plot-iris-logistic-py
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from sklearn.preprocessing import PolynomialFeatures,StandardScaler,LabelEncoder

In [2]:
# import some data to play with
iris = datasets.load_iris()
X_raw = iris.data[:, :2]  # we only take the first two features.
Y = iris.target

X_raw,Y = datasets.load_boston(return_X_y=True)  # we only take the first two features.
#preprocesado
X_raw = StandardScaler().fit_transform(X_raw)
lab_enc = LabelEncoder()
Y = lab_enc.fit_transform(Y)

X_train, X_test, y_train, y_test = train_test_split(X_raw, Y, test_size=.4, random_state=42)


poly = PolynomialFeatures(3, include_bias=True)
X_train = poly.fit_transform(X_train)
logreg = LogisticRegression(penalty='none',C=1e5, solver='lbfgs')

# we create an instance of Neighbours Classifier and fit the data.
logreg.fit(X_train, y_train)


LogisticRegression(C=100000.0, penalty='none')

In [9]:
error_test = []
error_sets = []
rounds = 5
logreg_test = LogisticRegression(penalty='none',C=1e5, solver='lbfgs', multi_class='multinomial')
for i in range(2,13):
    poly_test = PolynomialFeatures(i,include_bias=True)
    X_entrena, X_testeo, y_entrena, y_testeo = train_test_split(X_raw[:,:i], Y, test_size=0.3, random_state=42)
    X_entrena = poly_test.fit_transform(X_entrena)
    # we create an instance of Neighbours Classifier and fit the data.
    logreg_test.fit(X_entrena, y_entrena)
    X_testeo = poly_test.fit_transform(X_testeo)
    y_pred = logreg_test.predict(X_testeo)
    error_sets.append(1 - np.mean(y_pred == y_testeo))

plt.plot(error_test)
plt.plot(error_sets)
plt.show()

x_min, x_max = X_test[:, :].min() - .5, X_test[:, :].max() + .5
y_min, y_max = X_test[:, :].min() - .5, X_test[:, :].max() + .5
h = .02  # step size in the mesh
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
X_plot_raw = np.c_[xx.ravel(), yy.ravel()]
X_plot = poly.transform(X_plot_raw)


# Plot the decision boundary. For that, we will assign a color to each
# point in the mesh [x_min, x_max]x[y_min, y_max].

Z = logreg.predict(X_plot)

# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.figure(1, figsize=(4, 3))
plt.pcolormesh(xx, yy, Z, cmap=plt.cm.Paired)

# Plot also the training points
plt.scatter(X_test[:, 0], X_test[:, 1], c=y_test, edgecolors='k', cmap=plt.cm.Paired)
plt.xlabel('Sepal length')
plt.ylabel('Sepal width')

plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())
plt.xticks(())
plt.yticks(())

plt.show()

MemoryError: Unable to allocate 19.1 MiB for an array with shape (2509650,) and data type float64

In [ ]:
b) Lo que podemos pensar al ver que aumenta y después disminuye el error del modelo a medida que aumentamos la complejidad del modelo para el los datos de prueba, según lo dicho en "citar"  hay paraetros que no están aportando al sesgo inductivo, además de que para el primer descenso, el modelo esta sub pametrizando, hasta llegar al punto de interpolación cuando empieza una sobre pametrización de este. Esto pasa jutsto cuando en el set de rueba cambia el tamaño de uno de complejidad pequeño para la estimación, a uno con complejidad grande, por lo que al llegar al punto donde el error es cero en el entrenamiento, el aumento de complejidad para la prueba mejorar el rendimiento del modelo.

c) 

In [ ]:
### **Pregunta** 3

a)


  b)